In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio python-multipart
!pip install langchain langchain-community langchain-huggingface transformers accelerate bitsandbytes

In [ ]:
import os
os.system("fuser -k 8000/tcp")

256

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model_id = 'EleutherAI/polyglot-ko-1.3b'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
) .to(device)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.3,
    top_p=0.95,
    repetition_penalty=1.15
)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

game_context = """
[게임 개요]
- 이름: Asterisk (아스테리스크)
- 장르: 로그라이크 시뮬레이션 (Roguelike Simulation)
- 한 줄 소개: 기지 건설과 해체를 반복하며 최적의 효율을 찾아내는 우주 생존 게임.

[플레이어 경험 & 핵심 재미]
1. 간접 조작과 최적화: 플레이어는 유닛을 직접 조종하지 않습니다. 대신 지시를 내리면 유닛과 기지가 자동으로 움직입니다. 이들의 동선을 관리하고 생산 라인을 최적화하는 것이 핵심 재미입니다.
2. 전략적 생존과 확장: 제한된 자원과 공간 안에서 매번 달라지는 상황(로그라이크 요소)에 맞춰 최선의 선택을 내려야 합니다.
3. 독특한 건설 방식: 단순히 건물을 짓는 것을 넘어, '건물 조각'을 생산하고 이를 '조합'하거나 완성된 건물을 '분해'하며 기지를 유기적으로 성장시킵니다.

[차별화 포인트]
- 장르의 융합: 로그라이크의 '리플레이성/긴장감'과 시뮬레이션의 '건설/성장' 재미를 결합했습니다.
- 4대 자원 시스템: 4종의 기본 자원을 채굴하고 가공하여 기지를 확장하고, 기술 수준(테크트리)을 업그레이드합니다.
- 모선 성장 시스템: 행성에서 가공한 자원을 모선(Mothership)으로 전송하여 업그레이드할 수 있습니다. 이는 다음 탐사를 더 유리하게 만듭니다.

[핵심 게임 루프 (Core Loop)]
1. 탐사 준비: 시드 코어(Seed Core)를 준비하고 발사 지점을 선택하여 행성으로 코어를 발사합니다.
2. 기지 건설 (반복 단계):
   - 착륙 후 기초 기지를 건설합니다.
   - 자원 채굴 -> 건물 조각 생산 -> 건물 완성 -> 자원 가공 -> 기지 확장의 과정을 반복합니다.
   - 주기적으로 몰려오는 적들과 전투하며 기지를 방어합니다.
3. 탈출 및 복귀:
   - 목표 자원을 충분히 모으면 '시드 코어'를 시동합니다.
   - 코어가 가동되는 동안 최후의 전투(디펜스)를 치릅니다.
   - 방어에 성공하면 모선으로 복귀하며 탐사가 종료됩니다.

[플레이어의 목표와 선택]
- 궁극적 목표: 현재 필요한 자원을 효율적으로 채굴하여 시드 코어를 활성화하고 무사히 모선으로 복귀하는 것.
- 선택과 집중 (Risk vs Reward): "지금 바로 안전하게 출발할 것인가?" 아니면 "위험을 감수하고 자원을 더 캔 다음 출발할 것인가?"를 끊임없이 고민해야 합니다.
"""

chat_template = f"""
너는 게임 'Asterisk'의 공식 AI 가이드야.
아래의 [게임 정보]를 바탕으로 유저의 질문에 친절하고, 전문적이며, 약간은 위트 있게 답변해줘.
정보에 없는 내용은 지어내지 말고, "그 부분은 아직 비밀이에요!" 혹은 "개발자에게 문의해주세요."라고 답변해.

[게임 정보]
{game_context}

[대화 예시]
Q: 이 게임은 어떻게 하는 거야?
A: 유닛에게 명령을 내려서 자원을 캐고 기지를 짓는 게임이에요! 직접 조종하는 게 아니라 '최적화'를 해주는 게 핵심이죠. 🏭

Q: 게임의 목표가 뭐야?
A: 행성에서 자원을 모아 '시드 코어'를 작동시키고 모선으로 무사히 탈출하는 것입니다! 물론 그 과정에서 욕심을 부리다 죽을 수도 있죠. 😎

Q: {{question}}
A:"""

chat_prompt = PromptTemplate.from_template(chat_template)
chat_chain = chat_prompt | llm | StrOutputParser()

In [ ]:
analyze_template = """다음 리뷰의 감정을 '긍정' 또는 '부정'으로 분류하고, 0~100점 사이의 점수를 매기세요.

리뷰: 게임이 너무 어렵지만 재밌어요.
분석: 긍정, 80

리뷰: 버그가 너무 많아서 못해먹겠어요.
분석: 부정, 20

리뷰: {feedback}
분석:"""

analyze_chain = PromptTemplate.from_template(analyze_template) | llm | StrOutputParser()

In [ ]:
summary_template = """
다음 리뷰들을 읽고, 전체적인 여론을 한 문장으로 요약하고 핵심 키워드 3개를 뽑아주세요.

[리뷰 목록]
{reviews}

[분석 결과]
요약: 유저들은 전반적으로 난이도에 대해 불만이 있지만 그래픽은 칭찬합니다.
키워드: #난이도, #그래픽, #밸런스

[리뷰 목록]
{reviews_input}

[분석 결과]
"""
summary_prompt = PromptTemplate.from_template(summary_template)
summary_chain = summary_prompt | llm | StrOutputParser()

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import threading

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

ngrok.set_auth_token("3604L8FQC0MStT6pfNCWo9M9XJm_3oPnr4XouXPS6UUjgPdSt")

class ChatReq(BaseModel):
    message: str

class FeedbackReq(BaseModel):
    feedback: str

class BatchReq(BaseModel):
    feedbacks: List[str]

@app.post("/chat")
def chat(req: ChatReq):
    full_response = chat_chain.invoke({"question": req.message})
    print(f"🔍 AI 전체 응답 확인: {full_response}") # 로그로 확인용

    if "A:" in full_response:
        answer = full_response.rsplit("A:", 1)[-1].strip()
        if "Q:" in answer:
            answer = answer.split("Q:")[0].strip()
    else:
        answer = full_response

    return {"reply": answer}

@app.post("/analyze")
def analyze(req: FeedbackReq):
    res = analyze_chain.invoke({"feedback": req.feedback})
    try:
        parts = res.strip().split(',')
        sentiment = parts[0].strip()
        score = int(parts[1].strip())
    except:
        sentiment = "분석 중"
        score = 50
    return {"summary": req.feedback[:20], "sentiment_score": score, "sentiment_label": sentiment, "category": "일반"}

@app.post("/summarize")
def summarize(req: BatchReq):
    inputs = [r[:50] for r in req.feedbacks[-10:]]
    joined = "\n".join([f"- {r}" for r in inputs])

    res = summary_chain.invoke({
        "reviews": "- 그래픽이 좋아요\n- 너무 어려워요",
        "reviews_input": joined
    })

    print(f"DEBUG(LLM Output): {res}")

    summary_text = "데이터 분석에 실패했습니다."
    keywords = ["#분석실패"]

    try:
        lines = res.strip().split('\n')
        for line in lines:
            if "요약:" in line:
                summary_text = line.split("요약:")[1].strip()
            if "키워드:" in line:
                k_text = line.split("키워드:")[1].strip()
                keywords = [k.strip() for k in k_text.split(',')]
    except Exception as e:
        print(f"Parsing Error: {e}")

    if summary_text == "데이터 분석에 실패했습니다." and len(res) > 5:
         summary_text = res.replace("요약:", "").replace("키워드:", "")[:50] + "..."

    return {
        "overall_summary": summary_text,
        "top_keywords": keywords
    }

ngrok.kill()
ngrok_tunnel = ngrok.connect(8000)
print('🚀 Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()

🚀 Public URL: https://perioecic-edmond-unimitative.ngrok-free.dev


INFO:     Started server process [31139]
